In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import subprocess

In [12]:
#Função para Identificação de Eventos e Geração de Gráfico em um CSV
def identificarEventos(arquivo:str,TD:float,erro:float, grafico:bool=True)->pd.DataFrame:
    nome_base = arquivo.split('.')[0]
    #Rodar Tracker
    subprocess.run(['tracker.exe',arquivo, str(TD), str(erro)])
    #Ler arquivo de saída
    df_eventos = pd.read_csv(nome_base+"_eventos.csv")

    #Gerar Gráfico Caso Necessário
    if grafico:
        df = pd.read_csv(arquivo)
        df.rename(columns={'Tempo':'TEMPO','Pot. Ativa':'PA'}, inplace=True)
        df_eventos['tipo'] = df_eventos.apply(lambda x:1 if '1' in x['Evento'] else 2 if '2' in x['Evento'] else 3 if '3' in x['Evento'] else 4 if '4' in x['Evento'] else 0, axis=1)
        # Cores por tipo
        cores = {
            0: 'gray',
            1: 'red',
            2: 'green',
            3: 'orange',
            4: 'purple'
        }
        # Criando figura
        fig = go.Figure()

        # Curva de potência
        fig.add_trace(go.Scatter(
            x=df['TEMPO'],
            y=df['PA'],
            mode='lines',
            name='PA',
            line=dict(color='blue')
        ))

        # Barras verticais por tipo
        for tipo, cor in cores.items():
            sub = df_eventos[df_eventos['tipo'] == tipo]
            for tempo in sub['TimeStamp']:
                fig.add_trace(go.Scatter(
                    x=[tempo, tempo],
                    y=[df['PA'].min(), df['PA'].max()],
                    mode='lines',
                    line=dict(color=cor, dash='dash', width=1),
                    name=f'Tipo {tipo}',
                    showlegend=False  # Evita duplicar legenda
                ))

        # Legenda personalizada
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='gray'), name='OFF'))
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='red'), name='Tipo 1'))
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='green'), name='Tipo 2'))
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='orange'), name='Tipo 3'))
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='purple'), name='Tipo 4'))

        # Layout
        fig.update_layout(
            title='Reconhecimento de Eventos na Curva de Potência',
            xaxis_title='Tempo (s)',
            yaxis_title='Potência Ativa (W)',
            template='plotly_white',
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
        )
        fig.show()

In [15]:
identificarEventos('ventilador.csv', 300,1)

In [ ]:
identificarEventos('notebook.csv',300,1)